# 樊蓉槟
## 将获取到的车牌按照城市分组

In [ ]:
import socket
from collections import defaultdict
# 湖南省车牌前缀与城市对照表，映射到对应的城市名称
HUNAN_PLATES = {
    '湘A': '长沙', '湘B': '株洲', '湘C': '湘潭', '湘D': '衡阳',
    '湘E': '邵阳', '湘F': '岳阳', '湘G': '张家界', '湘H': '益阳',
    '湘J': '常德', '湘K': '娄底', '湘L': '郴州', '湘N': '怀化',
    '湘P': '湘西'
}
#初始化时设置服务器地址和端口，并初始化计数器。
class VehicleDataClient:
    def __init__(self, server_host, server_port):
        self.server_host = server_host # 服务器IP地址
        self.server_port = server_port # 服务器端口号
        self.city_counts = defaultdict(int) # 城市车辆计数器
        self.total_count = 0 # 总车辆计数器
    def process_vehicle_data(self, data):
        """处理车辆数据"""
        try:
            # 修正数据解析逻辑 - 现在数据格式是"车牌,时间"
            plate, _ = data.strip().split(',')  # 分割车牌和时间
            self.total_count += 1
            
            # 提取车牌前两位作为前缀，字符串切片获取地理标识
            prefix = plate[:2]
            
            # 判断是否为湖南车牌，确定归属地
            if prefix in HUNAN_PLATES:
                city = HUNAN_PLATES[prefix]
            else:
                city = '外省车辆'
         
            # 哈希表更新城市计数累积统计数据
            self.city_counts[city] += 1
            
            # 每10辆车打印一次当前统计
            if self.total_count % 10 == 0:
                self.print_statistics(intermediate=True)   
        except ValueError as e:
            print(f"⚠️ 数据格式错误: {data} | 错误详情: {str(e)}")
    #统计打印
    def print_statistics(self, intermediate=False):
        """打印统计信息"""
        title = "=== 当前统计 ===" if intermediate else "=== 最终统计报告 ==="
        print(f"\n{title}")
        print(f"📊 总接收车辆: {self.total_count}辆")
        print("\n🏙️ 按城市分组统计:")
        for city, count in sorted(self.city_counts.items(), key=lambda x: x[1], reverse=True):
            print(f"  {city.rjust(6)}: {str(count).rjust(4)}辆")
        print("===============" if intermediate else "===================")
    
    #数据获取
    def fetch_and_process_data(self, buffer_size=1024):
        """连接服务器并处理数据"""
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.connect((self.server_host, self.server_port))
                print(f"🔌 已连接到服务器 {self.server_host}:{self.server_port}")
                
                while True:
                    ## 接收数据（最大1024字节）
                    data = s.recv(buffer_size).decode('utf-8')
                    if not data:
                        break
                    # 处理可能的多条数据（以换行分隔）
                    for line in data.split('\n'):
                        if line.strip():  # 忽略空行
                            self.process_vehicle_data(line)      
            except ConnectionRefusedError:
                print(f"❌ 无法连接到服务器 {self.server_host}:{self.server_port}")
            except Exception as e:
                print(f"⚠️ 发生错误: {str(e)}")
            finally:
                self.print_statistics()
if __name__ == "__main__":
    # 配置服务器地址和端口
    SERVER_HOST = 'localhost'  # 替换为实际服务器IP
    SERVER_PORT = 9999         # 替换为实际服务器端口
    
    client = VehicleDataClient(SERVER_HOST, SERVER_PORT)
    print("🚗 车辆数据客户端启动，准备连接服务器...")
    client.fetch_and_process_data()